In [1]:
import os
import utils

import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.keras.models import load_model 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)

In [3]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [4]:
data_dir = 'moes_tavern_lines.txt'
text = utils.load_data(data_dir)

# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [5]:
view_sentence_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

Dataset Stats
Roughly the number of unique words: 11492


In [6]:
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))

Number of scenes: 262


In [7]:
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

Average number of sentences in each scene: 15.248091603053435


In [8]:
sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))

Number of lines: 4257


In [9]:
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

Average number of words in each line: 11.50434578341555


In [10]:
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [11]:
# The unique characters in the file

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

89 unique characters


In [12]:
# Vectorize the text

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [13]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  ...
}


In [14]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Moe_Szyslak: ' ---- characters mapped to int ---- > [40 69 59 54 46 80 79 73 66 55 65 25  1]


In [15]:
# Build Training Data

In [16]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

M
o
e
_
S


In [17]:
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello,"
" is Mike there? Last name, Rotch.\nMoe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike R"
'otch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?\nMoe_Szyslak: (INTO PHONE) Listen you litt'
"le puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice "
"pick.\nMoe_Szyslak: What's the matter Homer? You're not your normal effervescent self.\nHomer_Simpson: "


#### For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [19]:
# Print the first examples input and target values:

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello"
Target data: "oe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello,"


In [20]:
# For one charachter predict next one, then concatenate context through time

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 40 ('M')
  expected output: 69 ('o')
Step    1
  input: 69 ('o')
  expected output: 59 ('e')
Step    2
  input: 59 ('e')
  expected output: 54 ('_')
Step    3
  input: 54 ('_')
  expected output: 46 ('S')
Step    4
  input: 46 ('S')
  expected output: 80 ('z')


In [21]:
# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

dataset

<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [22]:
# Params

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape = [batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences = True,
                        stateful = True,
                        recurrent_initializer = 'glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [24]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim = embedding_dim,
  rnn_units = rnn_units,
  batch_size = BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           22784     
_________________________________________________________________
gru (GRU)                    (32, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (32, None, 89)            91225     
Total params: 4,052,313
Trainable params: 4,052,313
Non-trainable params: 0
_________________________________________________________________


In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 100, 89) # (batch_size, sequence_length, vocab_size)


In [26]:
# Check for one example

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy()

In [27]:
sampled_indices

array([ 7, 83,  7, 69, 43,  8, 69,  8, 10, 73, 79, 29, 33, 74,  1, 57, 86,
       81, 11, 28, 39, 54, 16, 62, 52, 77, 74, 39, 59, 17, 74, 14, 55, 30,
       37, 65, 55, 45, 53, 48, 45, 37, 21, 58, 65, 60, 74, 38, 84, 23, 58,
       88, 11, 87, 71, 27, 20,  0, 49, 27, 25, 57, 31, 73, 67, 20, 22, 41,
       59,  0,  3, 57, 88, 27, 83, 12, 27, 18, 78, 44, 50, 39,  1, 40, 15,
       38, 33, 75, 17, 73, 69, 81, 10, 61, 18, 39, 61,  5, 34, 63])

In [28]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "n, scram, so I can get ready!\nMoe_Szyslak: I'm tellin' ya, I'm fine. I've never been happier. See?\nM"

Next Char Predictions: 
 '&ã&oP\'o\')syBFt céÀ,AL_1hYwtLe2t/aCJkaRZURJ6dkftKä8dü,óq?5\nV?:cDsm57Ne\n"cü?ã-?3xQWL M0KFu2soÀ)g3Lg$Gi'


In [29]:
# Train

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (32, 100, 89)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.487979


In [30]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.001), loss = loss)

In [31]:
# Callbacks

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

In [32]:
EPOCHS = 100

In [33]:
history = model.fit(dataset, epochs = EPOCHS, callbacks = [checkpoint_callback])

Train for 94 steps
Epoch 1/100
94/94 [==============================] - 5s 55ms/step - loss: 3.1769
Epoch 2/100
94/94 [==============================] - 4s 45ms/step - loss: 2.1458
Epoch 3/100
94/94 [==============================] - 4s 47ms/step - loss: 1.9146
Epoch 4/100
94/94 [==============================] - 4s 45ms/step - loss: 1.7638
Epoch 5/100
94/94 [==============================] - 4s 43ms/step - loss: 1.6453
Epoch 6/100
94/94 [==============================] - 4s 45ms/step - loss: 1.5477
Epoch 7/100
94/94 [==============================] - 4s 45ms/step - loss: 1.4621
Epoch 8/100
94/94 [==============================] - 4s 47ms/step - loss: 1.3911
Epoch 9/100
94/94 [==============================] - 4s 45ms/step - loss: 1.3242
Epoch 10/100
94/94 [==============================] - 4s 45ms/step - loss: 1.2618
Epoch 11/100
94/94 [==============================] - 4s 46ms/step - loss: 1.2019
Epoch 12/100
94/94 [==============================] - 4s 46ms/step - loss: 1.1420
Epoch 

94/94 [==============================] - 4s 45ms/step - loss: 0.7561


#### To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [34]:
# Load Checkpoint

tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [35]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            22784     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 89)             91225     
Total params: 4,052,313
Trainable params: 4,052,313
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Get Pred Func

def generate_text(model, start_string):
  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [38]:
print(generate_text(model, start_string = u"Moe_Szyslak: "))

Moe_Szyslak: You moro again Lenny sixce the Simpson.
Moe_Szyslak: What'sa!
Skalleeth them, I'm cleater then we just head down 'els of somethin' food?
Moe_Szyslak: Don't worry Plastical, better shurtimpson: know what I'm doing. XCRUTINING) The Beave Won't.
Youncelice, restayits and the rest of the next thing for me.
Bumble: No.
Homer_Simpson: (DERISTO HARD) FALL SOBIRS, MAVES TATTHY / INSTER DAY/ TO PLAYE DE PETHY-NG.....
Barney_Gumble: I'm doing it! Here's "Walthers: own a beer of the Gumber You guys. You and gover sational cripson: (MEANSIGUA) Good nice problems.
V.ING HOMER) I know how you see hobod? (LIPTONG TO BOOK)
Moe_Szyslak: So, uh, what's sk...
Homer_Simpson: (TOUSTEE EMBURDS) Oh boe, is turning, you were's actually everyone thing you're fat champiest guy. Moe, a million dol.
Moe_Szyslak: Why don't you? 'Cause go to pether than men in or stool.
Homer_Simpson: Oh! / EAKING A BIGRIANOFBIRD LY BUT TOD THE BALKING UP WITH LOOKS) Kmoke we saved my like!


Moe_Szyslak: So, I can't--